# Procesamiento Digital de Señales <img src="./logo_UTN.svg" align="right" width="150"/>
## Alumno: Emanuel Olay
### Tarea Semanal: 3
### Consigna: 
En esta tarea simularemos el comportamiento de un ADC y cómo sus parámetros, tamaño de palabra de B bits y frecuencia de muestreo $f_s$ afectan a una señal (aproximadamente) analógica.

Para ello se puede analizar una senoidal con los siguientes parámetros:

frecuencia $f_0$ arbitraria, por ejemplo $f_0=f_S/N=\Delta f$, 
Amplitud normalizada, es decir unitaria
Se pide diseñar un bloque cuantizador que opere sobre una señal discreta en tiempo $s_R$, de forma tal que para un ADC de B bits y rango ±VF, el operador.

$$s_Q=Q_{B,VF}(s_R)$$


generará una $s_Q$ comprendida entre $±VF$ y con valores discretos establecidos por el paso de cuantización $q=\frac{2.V_F}{2^B−1}\approx \frac{V_F}{2^{B−1}} $(Volts).

Visualice en una misma gráfica $s_R$ y $s_Q$, donde se pueda observar que tienen el mismo rango en Volts y el efecto de la cuantización para $V_F=2$ Volts y  $B = 4, 8$ y $16$ bits.

In [1]:
%reset
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import scipy.stats as stats
import matplotlib.patches as mpatches
from scipy.fft import fft
%matplotlib

plt.ion
plt.rcParams['figure.figsize'] = (13,6)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Using matplotlib backend: Qt5Agg


In [2]:
N = 300 # muestras
fs= 500 #Hz
B = [4,8,16] # bits
V_f=2 # volts
## Declaro los pasos para cada bit distinto
q = [0]*len(B)
for i in range(len(B)):
    q[i] = V_f/2**(B[i]-1)


##vector tiempo de 0 a 1, no incluido, con N saltos
t = np.linspace(0, 1, N, endpoint=False)

##Noise
noise=[[0]*N]*len(B)
for i in range(len(B)):
    noise[i] = np.random.normal(0, q[i]/2, len(t))

#senoidal de amplitud 1 y 1 hz de frecuencia
amp=1 #amplitud en volts
freq=1 #fs/N #frecuencia en Hz
fase=0 #radianes
seno_cont=[[0]*N]*len(B)
for i in range(len(B)):
    seno_cont[i] = amp*np.sin(2 * np.pi * freq * t + fase) + noise[i]


e=[0]*len(B)
corr=[0]*len(B)
valor_medio=[0]*len(B)
muestreada=[[0]*N]*len(B)
DEP=[[0]*N]*len(B)

In [3]:
##Analizo el ruido
for i in range(len(B)):
    DEP[i]=np.abs(fft(noise[i]))**2
    #energia_freq= sum(np.abs(noise[i])**2)

In [4]:
for i in range(len(B)):
    muestreada[i]=np.round(seno_cont[i]/q[i])*q[i]
    plt.figure(i)
    plt.plot(t,seno_cont[i])
    plt.stem(t, muestreada[i])
    plt.title('Continua vs Muestreada con Bits='+str(B[i]))
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Volts [V]')
    notita = mpatches.Patch(label='Resolucion:'+str(q[i]))
    plt.legend(handles=[notita])

In [5]:
for i in range(len(B)):
    e[i]=seno_cont[i]-muestreada[i]
    plt.figure('1-'+str(i))
    plt.stem(t,e[i])
    plt.title('Error'+str(i))
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Volts [V]')

In [6]:
# for i in range(len(B)):
#     nobs, minmax, mean, variance, skewness, kurtosis=stats.describe(e[i])
#     print('Media     [',i,']=',mean)
#     print('Varianza  [',i,']=',variance)

In [7]:
for i in range(len(B)):
    corr[i] = sig.correlate(e[i], e[i], mode='same')
    plt.figure('2-'+str(i))
    plt.stem(t,corr[i])
    plt.title('Correlacion Error'+str(i)+'-Error'+str(i))
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Volts [V]')

In [8]:
for i in range(len(B)):
    plt.figure('3-'+str(i))
    count, bins, ignored=plt.hist(e[i])
    plt.title('Histograma del Error')
    plt.xlabel('Valor de la variable')
    plt.ylabel('Conteo')

In [9]:
for i in range(len(B)):
    plt.figure('4-'+str(i))
    plt.stem(t,DEP[i])
    plt.title('Densisdad Espectral de potencia del Ruido')
    plt.xlabel('Frecuencia [hz]')
    plt.ylabel('Watts [W]')

In [10]:
for i in range(len(B)):
    corr[i] = sig.correlate(noise[i], noise[i], mode='same')
    plt.figure('5-'+str(i))
    plt.stem(t,corr[i])
    plt.title('Correlacion Ruido')
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Volts [V]')
    pot_tot=np.sum(DEP[i])/N
    print('potencia total=',pot_tot)

potencia total= 5.774644060525166
potencia total= 0.017184603592370753
potencia total= 2.783611786252761e-07
